In [1]:
import csv
import networkx as nx
import pprint as pp

In [2]:
def compute_good_local_community(graph, seed_node_id, alpha=0.9):
    #
    # Creation of the teleporting probability distribution for the selected node...
    map_teleporting_probability_distribution__node_id__probability = {}
    for node_id in graph:
        map_teleporting_probability_distribution__node_id__probability[node_id] = 0.
    map_teleporting_probability_distribution__node_id__probability[seed_node_id] = 1.
    #
    # Computation of the PageRank vector.
    map__node_id__node_pagerank_value = nx.pagerank(graph, alpha=alpha,
                                                    personalization=map_teleporting_probability_distribution__node_id__probability)
    #
    # Put all nodes in a list and sort the list in descending order of the “normalized_score”.
    sorted_list__node_id__normalized_score = [(node_id, score / graph.degree[node_id])
                                              for node_id, score in map__node_id__node_pagerank_value.items()]
    sorted_list__node_id__normalized_score.sort(key=lambda x: (-x[1], x[0]))
    #
    # LET'S SWEEP!
    index_representing_the_set_of_node_ids_with_maximum_conductance = -1
    min_conductance_value = float("+inf")
    set__node_ids_in_the_candidate_community = set()
    set__node_ids_in_the_COMPLEMENT_of_the_candidate_community_to_the_entire_set_of_nodes = set(graph.nodes())
    for sweep_index in range(0, len(sorted_list__node_id__normalized_score) - 1):
        #
        # Creation of the set of nodes representing the candidate community and
        # its complement to the entire set of nodes in the graph.
        current_node_id = sorted_list__node_id__normalized_score[sweep_index][0]
        set__node_ids_in_the_candidate_community.add(current_node_id)
        set__node_ids_in_the_COMPLEMENT_of_the_candidate_community_to_the_entire_set_of_nodes.remove(current_node_id)
        #
        # Evaluation of the quality of the candidate community according to its conductance value.
        conductance_value = nx.algorithms.cuts.conductance(graph,
                                                           set__node_ids_in_the_candidate_community,
                                                           set__node_ids_in_the_COMPLEMENT_of_the_candidate_community_to_the_entire_set_of_nodes)
        #
        # Discard local communities with conductance 0 or 1.
        if conductance_value == 0. or conductance_value == 1.:
            continue

        ###### PART ADDED FROM ORIGINAL CODE ######
        # Discard local communities with more than 140 Pokemons
        if len(set__node_ids_in_the_candidate_community) > 140:
            continue
        ################### END ###################

        #
        # Update the values of variables representing the best solution generated so far.
        if conductance_value < min_conductance_value:
            min_conductance_value = conductance_value
            index_representing_the_set_of_node_ids_with_maximum_conductance = sweep_index
    #
    # Creation of the set of nodes representing the best local community generated by the sweeping procedure.
    set__node_ids_with_minimum_conductance = set([node_id for node_id, normalized_score in
                                                  sorted_list__node_id__normalized_score[
                                                  :index_representing_the_set_of_node_ids_with_maximum_conductance + 1]])
    #
    return set__node_ids_with_minimum_conductance, min_conductance_value

In [3]:
# graph creation function
def graph_create_from_tsv(tsvFilePath):
    # reading tsv file
    input_file_handler = open(tsvFilePath, 'r', encoding="utf-8")
    tsv_reader = csv.reader(input_file_handler, delimiter='\t')
    tsv_reader.__next__()
    list_adj = []
    # inserting adjacent pokemon couples in a list
    for pokemon in tsv_reader:
        pokemon1 = pokemon[0]
        pokemon2 = pokemon[1]
        list_adj.append((pokemon1, pokemon2))
    input_file_handler.close()

    #creating graph
    graph = nx.Graph()
    graph.add_edges_from(list_adj)
    return graph


In [4]:
damp_val = [0.95, 0.9, 0.85, 0.8, 0.75, 0.7, 0.65, 0.6, 0.55,
            0.5, 0.45, 0.4, 0.35, 0.3, 0.25, 0.2, 0.15, 0.1, 0.05]

graph = graph_create_from_tsv('dataset/pkmn_graph_data.tsv')
output_data = 'dataset/pkmn_output.tsv'

pkmn_nodes = list(graph.nodes)
# dictionary to store the frequency of a pokemon in a community
pkmn_freq = dict.fromkeys(list(graph.nodes), 0)

# list to store the rows needed
# (pokemon_name, number_of_nodes_in_the_local_community, conductance_value_of_the_local_community)
res = []
for perc, pokemon in enumerate(pkmn_nodes):
    # how much has been done
    print("{:.2%}".format(perc / len(pkmn_nodes)))
    set_local_comm = []
    for alpha in damp_val:
        set_local_comm.append(compute_good_local_community(graph, pokemon, alpha=alpha))

    # x = [(set__node_ids_with_minimum_conductance, min_conductance_value)]
    # x[0] = set__node_ids_with_minimum_conductance
    # x[1] = min_conductance_value
    best_min_cond_val = min(set_local_comm, key=lambda x: x[1])
    res.append([pokemon, len(best_min_cond_val[0]), best_min_cond_val[1]])

    # calculating frequency of a pokemon in a community, for the second part of part_2_2
    for pkmn in best_min_cond_val[0]:
        pkmn_freq[pkmn] += 1

0.00%
0.35%
0.70%
1.05%
1.39%
1.74%
2.09%
2.44%
2.79%
3.14%
3.48%
3.83%
4.18%
4.53%
4.88%
5.23%
5.57%
5.92%
6.27%
6.62%
6.97%
7.32%
7.67%
8.01%
8.36%
8.71%
9.06%
9.41%
9.76%
10.10%
10.45%
10.80%
11.15%
11.50%
11.85%
12.20%
12.54%
12.89%
13.24%
13.59%
13.94%
14.29%
14.63%
14.98%
15.33%
15.68%
16.03%
16.38%
16.72%
17.07%
17.42%
17.77%
18.12%
18.47%
18.82%
19.16%
19.51%
19.86%
20.21%
20.56%
20.91%
21.25%
21.60%
21.95%
22.30%
22.65%
23.00%
23.34%
23.69%
24.04%
24.39%
24.74%
25.09%
25.44%
25.78%
26.13%
26.48%
26.83%
27.18%
27.53%
27.87%
28.22%
28.57%
28.92%
29.27%
29.62%
29.97%
30.31%
30.66%
31.01%
31.36%
31.71%
32.06%
32.40%
32.75%
33.10%
33.45%
33.80%
34.15%
34.49%
34.84%
35.19%
35.54%
35.89%
36.24%
36.59%
36.93%
37.28%
37.63%
37.98%
38.33%
38.68%
39.02%
39.37%
39.72%
40.07%
40.42%
40.77%
41.11%
41.46%
41.81%
42.16%
42.51%
42.86%
43.21%
43.55%
43.90%
44.25%
44.60%
44.95%
45.30%
45.64%
45.99%
46.34%
46.69%
47.04%
47.39%
47.74%
48.08%
48.43%
48.78%
49.13%
49.48%
49.83%
50.17%
50.52%
50.87%


In [5]:
# writing tsv file to store the results
print('\n##############################')
print('###### Writing tsv file ######')
print('##############################\n')
res_alpha_order = list(sorted(res, key=lambda x: x[0]))
f = open(output_data, 'w')
f.write("pokemon_name\tnumber_of_nodes_in_the_local_community\tconductance_value_of_the_local_community" + "\n")
for row in res_alpha_order:
	f.write(str(row[0]) + "\t" + str(row[1]) + "\t" + str(row[2]) + "\n")
f.close()
print('############ DONE ############')


##############################
###### Writing tsv file ######
##############################

############ DONE ############


In [6]:
# calculating high and low frequency of a pokemon
sort_dict_desc = dict(sorted(pkmn_freq.items(), key=lambda x: x[1], reverse=True))
dict_len = len(sort_dict_desc)

print('\n##########################################')
print('###### Top 5 most frequent Pokemon  ######')
print('##########################################\n')
print('Pokemon\tFrequency\n')
for count, pokemon in enumerate(sort_dict_desc):
    if count >= 5:
        break
    print(str(pokemon) + '\t' + str(sort_dict_desc[pokemon]) + '\n')

print('\n###########################################')
print('###### Top 5 least frequent Pokemon  ######')
print('###########################################\n')

for count, pokemon in enumerate(sort_dict_desc):
    if count < (dict_len-5):
        continue
    print(str(pokemon) + '\t' + str(sort_dict_desc[pokemon]) + '\n')



##########################################
###### Top 5 most frequent Pokemon  ######
##########################################

Pokemon	Frequency

Kartana	178

Incineroar	177

Primarina	172

Mimikyu	170

Volcarona	169


###########################################
###### Top 5 least frequent Pokemon  ######
###########################################

Roserade	43

Miltank	42

Magnezone	37

Hitmonlee	37

Armaldo	31

